In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from typing import Tuple, List

# Configuração para gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

## 1. Ambiente com Paredes e Penalização

In [ ]:
class GridEnvironmentWithWalls:
    """Ambiente de grelha 10x10 com paredes e penalização."""
    
    def __init__(self):
        self.grid_size = 10
        self.n_states = 100
        self.goal_state = 100
        self.actions = ['UP', 'DOWN', 'LEFT', 'RIGHT']
        self.n_actions = len(self.actions)
        
        # Paredes conforme Figura 4
        self.walls = {4, 14, 24, 34, 44, 54, 64, 74, 84, 
                     17, 27, 37, 47, 57, 67, 77, 87, 97}
        
        # Penalização por bater em parede
        self.wall_penalty = -0.1
        self.goal_reward = 100.0
        
    def state_to_position(self, state: int) -> Tuple[int, int]:
        """Converte estado (1-100) para posição (linha, coluna)."""
        row = (state - 1) // self.grid_size
        col = (state - 1) % self.grid_size
        return row, col
    
    def position_to_state(self, row: int, col: int) -> int:
        """Converte posição (linha, coluna) para estado (1-100)."""
        return row * self.grid_size + col + 1
    
    def is_valid_state(self, state: int) -> bool:
        """Verifica se o estado está dentro da grelha e não é parede."""
        if state < 1 or state > self.n_states:
            return False
        return state not in self.walls
    
    def transition(self, state: int, action: str) -> Tuple[int, float]:
        """Executa ação e retorna (novo_estado, recompensa)."""
        row, col = self.state_to_position(state)
        
        # Calcula novo estado baseado na ação
        if action == 'UP':
            new_row, new_col = row - 1, col
        elif action == 'DOWN':
            new_row, new_col = row + 1, col
        elif action == 'LEFT':
            new_row, new_col = row, col - 1
        elif action == 'RIGHT':
            new_row, new_col = row, col + 1
        else:
            raise ValueError(f"Ação inválida: {action}")
        
        # Verifica se novo estado é válido
        if (new_row < 0 or new_row >= self.grid_size or 
            new_col < 0 or new_col >= self.grid_size):
            # Fora dos limites - permanece no mesmo estado com penalização
            return state, self.wall_penalty
        
        new_state = self.position_to_state(new_row, new_col)
        
        # Verifica se novo estado é parede
        if new_state in self.walls:
            # Bate em parede - permanece no mesmo estado com penalização
            return state, self.wall_penalty
        
        # Movimento válido
        reward = self.goal_reward if new_state == self.goal_state else 0.0
        return new_state, reward
    
    def get_reward(self, state: int) -> float:
        """Retorna recompensa do estado."""
        return self.goal_reward if state == self.goal_state else 0.0
    
    def random_action(self) -> str:
        """Retorna ação aleatória."""
        return np.random.choice(self.actions)
    
    def visualize_environment(self):
        """Visualiza o ambiente com paredes."""
        grid = np.zeros((self.grid_size, self.grid_size))
        
        # Marca paredes
        for wall in self.walls:
            row, col = self.state_to_position(wall)
            grid[row, col] = -1
        
        # Marca objetivo
        goal_row, goal_col = self.state_to_position(self.goal_state)
        grid[goal_row, goal_col] = 2
        
        plt.figure(figsize=(10, 10))
        plt.imshow(grid, cmap='RdYlGn', interpolation='nearest')
        plt.colorbar(label='Tipo de célula', ticks=[-1, 0, 2])
        plt.gca().set_yticklabels([])
        plt.gca().set_xticklabels([])
        
        # Adiciona números dos estados
        for state in range(1, self.n_states + 1):
            row, col = self.state_to_position(state)
            color = 'white' if state in self.walls else 'black'
            plt.text(col, row, str(state), ha='center', va='center', 
                    fontsize=8, color=color, weight='bold')
        
        plt.title('Ambiente com Paredes (Figura 4)\n-1: Parede, 0: Livre, 2: Objetivo', 
                 fontsize=14, weight='bold')
        plt.tight_layout()
        plt.show()

In [ ]:
# Visualizar o ambiente
env = GridEnvironmentWithWalls()
env.visualize_environment()

## 2. Baseline: Random Walk com Paredes

In [ ]:
def run_random_walk_episode(env: GridEnvironmentWithWalls, 
                            initial_state: int = 1, 
                            max_steps: int = 1000) -> Tuple[float, int, float]:
    """Executa um episódio com random walk.
    
    Retorna:
        - recompensa média por passo
        - número de passos até objetivo (ou max_steps)
        - tempo de execução
    """
    start_time = time.time()
    
    state = initial_state
    total_reward = 0.0
    steps = 0
    
    for step in range(max_steps):
        action = env.random_action()
        new_state, reward = env.transition(state, action)
        
        total_reward += reward
        steps += 1
        
        if new_state == env.goal_state:
            break
        
        state = new_state
    
    execution_time = time.time() - start_time
    avg_reward_per_step = total_reward / steps if steps > 0 else 0.0
    
    return avg_reward_per_step, steps, execution_time

In [ ]:
# Executar baseline com paredes
np.random.seed(42)
n_episodes = 30

baseline_results = {
    'avg_rewards': [],
    'steps': [],
    'times': []
}

print("Executando baseline (Random Walk) com paredes...")
for episode in range(n_episodes):
    avg_reward, steps, exec_time = run_random_walk_episode(env)
    baseline_results['avg_rewards'].append(avg_reward)
    baseline_results['steps'].append(steps)
    baseline_results['times'].append(exec_time)
    
    if (episode + 1) % 10 == 0:
        print(f"Episódio {episode + 1}/{n_episodes} concluído")

# Estatísticas
print("\n" + "="*60)
print("BASELINE COM PAREDES - ESTATÍSTICAS")
print("="*60)
print(f"Recompensa média por passo: {np.mean(baseline_results['avg_rewards']):.4f} ± {np.std(baseline_results['avg_rewards']):.4f}")
print(f"Número de passos: {np.mean(baseline_results['steps']):.2f} ± {np.std(baseline_results['steps']):.2f}")
print(f"Tempo de execução: {np.mean(baseline_results['times']):.4f} ± {np.std(baseline_results['times']):.4f} s")
print("="*60)

In [ ]:
# Visualização dos resultados baseline
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].boxplot(baseline_results['avg_rewards'])
axes[0].set_title('Recompensa Média por Passo\n(Baseline com Paredes)', weight='bold')
axes[0].set_ylabel('Recompensa Média')
axes[0].grid(True, alpha=0.3)

axes[1].boxplot(baseline_results['steps'])
axes[1].set_title('Número de Passos\n(Baseline com Paredes)', weight='bold')
axes[1].set_ylabel('Passos')
axes[1].grid(True, alpha=0.3)

axes[2].boxplot(baseline_results['times'])
axes[2].set_title('Tempo de Execução\n(Baseline com Paredes)', weight='bold')
axes[2].set_ylabel('Tempo (s)')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Q-Learning com Paredes

In [ ]:
class QLearningAgentWithWalls:
    """Agente Q-Learning para ambiente com paredes."""
    
    def __init__(self, env: GridEnvironmentWithWalls, alpha: float = 0.7, gamma: float = 0.99):
        self.env = env
        self.alpha = alpha
        self.gamma = gamma
        
        # Tabela Q: [estado, ação]
        self.Q = np.zeros((env.n_states + 1, env.n_actions))
        
    def get_action_index(self, action: str) -> int:
        """Converte ação para índice."""
        return self.env.actions.index(action)
    
    def get_best_action(self, state: int) -> str:
        """Retorna melhor ação para o estado (com desempate aleatório)."""
        q_values = self.Q[state]
        max_q = np.max(q_values)
        best_actions = [i for i, q in enumerate(q_values) if q == max_q]
        best_action_idx = np.random.choice(best_actions)
        return self.env.actions[best_action_idx]
    
    def update_q(self, state: int, action: str, reward: float, next_state: int):
        """Atualiza valor Q."""
        action_idx = self.get_action_index(action)
        current_q = self.Q[state, action_idx]
        max_next_q = np.max(self.Q[next_state])
        
        # Q-learning update
        new_q = (1 - self.alpha) * current_q + self.alpha * (reward + self.gamma * max_next_q)
        self.Q[state, action_idx] = new_q
    
    def train_random_walk(self, n_steps: int, test_points: List[int], 
                          initial_state: int = 1) -> dict:
        """Treino com random walk."""
        results = {
            'test_points': test_points,
            'test_rewards': []
        }
        
        state = initial_state
        test_idx = 0
        
        for step in range(1, n_steps + 1):
            # Ação aleatória
            action = self.env.random_action()
            next_state, reward = self.env.transition(state, action)
            
            # Atualiza Q
            self.update_q(state, action, reward, next_state)
            
            # Reset se chegou ao objetivo
            if next_state == self.env.goal_state:
                state = initial_state
            else:
                state = next_state
            
            # Teste nos pontos especificados
            if test_idx < len(test_points) and step == test_points[test_idx]:
                test_reward = self.test_policy(n_steps=1000, initial_state=initial_state)
                results['test_rewards'].append(test_reward)
                test_idx += 1
        
        return results
    
    def train_epsilon_greedy(self, n_steps: int, test_points: List[int], 
                            greed: float = 0.9, initial_state: int = 1) -> dict:
        """Treino com estratégia ε-greedy."""
        results = {
            'test_points': test_points,
            'test_rewards': []
        }
        
        state = initial_state
        test_idx = 0
        
        for step in range(1, n_steps + 1):
            # Escolhe ação: greedy ou aleatória
            if np.random.random() < greed:
                action = self.get_best_action(state)
            else:
                action = self.env.random_action()
            
            next_state, reward = self.env.transition(state, action)
            
            # Atualiza Q
            self.update_q(state, action, reward, next_state)
            
            # Reset se chegou ao objetivo
            if next_state == self.env.goal_state:
                state = initial_state
            else:
                state = next_state
            
            # Teste nos pontos especificados
            if test_idx < len(test_points) and step == test_points[test_idx]:
                test_reward = self.test_policy(n_steps=1000, initial_state=initial_state)
                results['test_rewards'].append(test_reward)
                test_idx += 1
        
        return results
    
    def test_policy(self, n_steps: int = 1000, initial_state: int = 1) -> float:
        """Testa política atual sem atualizar Q."""
        state = initial_state
        total_reward = 0.0
        steps = 0
        
        for _ in range(n_steps):
            action = self.get_best_action(state)
            next_state, reward = self.env.transition(state, action)
            
            total_reward += reward
            steps += 1
            
            if next_state == self.env.goal_state:
                break
            
            state = next_state
        
        return total_reward / steps if steps > 0 else 0.0
    
    def get_utility_map(self) -> np.ndarray:
        """Retorna mapa de utilidade máxima por estado."""
        utility = np.zeros((self.env.grid_size, self.env.grid_size))
        
        for state in range(1, self.env.n_states + 1):
            row, col = self.env.state_to_position(state)
            if state in self.env.walls:
                utility[row, col] = np.nan
            else:
                utility[row, col] = np.max(self.Q[state])
        
        return utility
    
    def visualize_utility(self, title: str = "Mapa de Utilidade"):
        """Visualiza mapa de utilidade."""
        utility = self.get_utility_map()
        
        plt.figure(figsize=(12, 10))
        im = plt.imshow(utility, cmap='viridis', interpolation='nearest')
        plt.colorbar(im, label='Utilidade Máxima')
        
        # Adiciona valores
        for state in range(1, self.env.n_states + 1):
            row, col = self.env.state_to_position(state)
            if state not in self.env.walls:
                max_q = np.max(self.Q[state])
                plt.text(col, row, f'{max_q:.1f}', ha='center', va='center',
                        fontsize=8, color='white' if max_q > utility[~np.isnan(utility)].mean() else 'black')
        
        plt.title(title, fontsize=14, weight='bold')
        plt.tight_layout()
        plt.show()

## 4. Experimento: Random Walk com Paredes

In [ ]:
# Parâmetros
n_experiments = 30
n_training_steps = 20000
test_points = [100, 200, 500, 600, 700, 800, 900, 1000, 
               2500, 5000, 7500, 10000, 12500, 15000, 17500, 20000]

# Armazenar resultados
random_walk_results = {
    'test_rewards': np.zeros((n_experiments, len(test_points))),
    'execution_times': []
}

print("Executando Q-Learning com Random Walk (com paredes)...")
for exp in range(n_experiments):
    np.random.seed(exp)
    
    agent = QLearningAgentWithWalls(env)
    start_time = time.time()
    
    results = agent.train_random_walk(n_training_steps, test_points)
    
    exec_time = time.time() - start_time
    random_walk_results['test_rewards'][exp] = results['test_rewards']
    random_walk_results['execution_times'].append(exec_time)
    
    if (exp + 1) % 5 == 0:
        print(f"Experimento {exp + 1}/{n_experiments} concluído (tempo: {exec_time:.2f}s)")

# Estatísticas
print("\n" + "="*60)
print("Q-LEARNING RANDOM WALK COM PAREDES - ESTATÍSTICAS")
print("="*60)
print(f"Tempo médio de execução: {np.mean(random_walk_results['execution_times']):.2f} ± {np.std(random_walk_results['execution_times']):.2f} s")
print(f"Recompensa final (20000 passos): {np.mean(random_walk_results['test_rewards'][:, -1]):.4f} ± {np.std(random_walk_results['test_rewards'][:, -1]):.4f}")
print("="*60)

In [ ]:
# Gráfico de evolução
mean_rewards = np.mean(random_walk_results['test_rewards'], axis=0)
std_rewards = np.std(random_walk_results['test_rewards'], axis=0)

plt.figure(figsize=(14, 6))
plt.plot(test_points, mean_rewards, 'o-', linewidth=2, markersize=8, label='Média')
plt.fill_between(test_points, 
                 mean_rewards - std_rewards, 
                 mean_rewards + std_rewards, 
                 alpha=0.3, label='± 1 desvio padrão')
plt.xlabel('Passos de Treino', fontsize=12, weight='bold')
plt.ylabel('Recompensa Média por Passo (Teste)', fontsize=12, weight='bold')
plt.title('Evolução do Q-Learning com Random Walk (Com Paredes)', fontsize=14, weight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:
# Visualizar mapa de utilidade (última experiência)
agent.visualize_utility("Mapa de Utilidade - Q-Learning Random Walk (Com Paredes)")

## 5. Experimento: ε-Greedy com Paredes

In [ ]:
# Testar diferentes valores de greed
greed_values = [0.2, 0.5, 0.9]

epsilon_greedy_results = {}

for greed in greed_values:
    print(f"\n{'='*60}")
    print(f"Testando ε-greedy com greed = {greed}")
    print(f"{'='*60}")
    
    results = {
        'test_rewards': np.zeros((n_experiments, len(test_points))),
        'execution_times': []
    }
    
    for exp in range(n_experiments):
        np.random.seed(exp)
        
        agent = QLearningAgentWithWalls(env)
        start_time = time.time()
        
        exp_results = agent.train_epsilon_greedy(n_training_steps, test_points, greed=greed)
        
        exec_time = time.time() - start_time
        results['test_rewards'][exp] = exp_results['test_rewards']
        results['execution_times'].append(exec_time)
        
        if (exp + 1) % 10 == 0:
            print(f"Experimento {exp + 1}/{n_experiments} concluído")
    
    epsilon_greedy_results[greed] = results
    
    # Estatísticas
    print(f"\nTempo médio: {np.mean(results['execution_times']):.2f} ± {np.std(results['execution_times']):.2f} s")
    print(f"Recompensa final: {np.mean(results['test_rewards'][:, -1]):.4f} ± {np.std(results['test_rewards'][:, -1]):.4f}")

In [ ]:
# Comparar todas as estratégias
plt.figure(figsize=(16, 6))

# Random Walk
mean_rw = np.mean(random_walk_results['test_rewards'], axis=0)
std_rw = np.std(random_walk_results['test_rewards'], axis=0)
plt.plot(test_points, mean_rw, 'o-', linewidth=2, markersize=6, label='Random Walk')
plt.fill_between(test_points, mean_rw - std_rw, mean_rw + std_rw, alpha=0.2)

# ε-greedy
colors = ['red', 'green', 'blue']
for i, greed in enumerate(greed_values):
    mean_eg = np.mean(epsilon_greedy_results[greed]['test_rewards'], axis=0)
    std_eg = np.std(epsilon_greedy_results[greed]['test_rewards'], axis=0)
    plt.plot(test_points, mean_eg, 's-', linewidth=2, markersize=6, 
             label=f'ε-greedy (greed={greed})', color=colors[i])
    plt.fill_between(test_points, mean_eg - std_eg, mean_eg + std_eg, alpha=0.2, color=colors[i])

plt.xlabel('Passos de Treino', fontsize=12, weight='bold')
plt.ylabel('Recompensa Média por Passo', fontsize=12, weight='bold')
plt.title('Comparação de Estratégias (Ambiente com Paredes)', fontsize=14, weight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=10, loc='best')
plt.tight_layout()
plt.show()

In [ ]:
# Visualizar mapas de utilidade para diferentes estratégias
fig, axes = plt.subplots(1, 4, figsize=(24, 6))

strategies = [
    ('Random Walk', None, random_walk_results),
    ('ε-greedy (0.2)', 0.2, epsilon_greedy_results[0.2]),
    ('ε-greedy (0.5)', 0.5, epsilon_greedy_results[0.5]),
    ('ε-greedy (0.9)', 0.9, epsilon_greedy_results[0.9])
]

for idx, (name, greed, _) in enumerate(strategies):
    np.random.seed(10)
    agent = QLearningAgentWithWalls(env)
    
    if greed is None:
        agent.train_random_walk(n_training_steps, test_points)
    else:
        agent.train_epsilon_greedy(n_training_steps, test_points, greed=greed)
    
    utility = agent.get_utility_map()
    
    im = axes[idx].imshow(utility, cmap='viridis', interpolation='nearest')
    axes[idx].set_title(name, fontsize=12, weight='bold')
    plt.colorbar(im, ax=axes[idx], fraction=0.046, pad=0.04)

plt.suptitle('Mapas de Utilidade - Comparação de Estratégias (Com Paredes)', 
             fontsize=14, weight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 6. Comparação com Resultados Anteriores (Sem Paredes)

### Análise Comparativa

Para comparar adequadamente com os exercícios anteriores (sem paredes), vamos analisar:

1. **Impacto das paredes no baseline (random walk)**
2. **Eficácia do Q-Learning com paredes vs. sem paredes**
3. **Efeito da penalização na aprendizagem**
4. **Diferenças nos mapas de utilidade**

In [ ]:
# Resumo comparativo
print("\n" + "="*80)
print("RESUMO COMPARATIVO: AMBIENTE COM PAREDES vs SEM PAREDES")
print("="*80)

print("\n1. BASELINE (Random Walk)")
print("-" * 80)
print(f"   Recompensa média por passo: {np.mean(baseline_results['avg_rewards']):.4f} ± {np.std(baseline_results['avg_rewards']):.4f}")
print(f"   Número médio de passos: {np.mean(baseline_results['steps']):.2f} ± {np.std(baseline_results['steps']):.2f}")
print(f"   Tempo médio: {np.mean(baseline_results['times']):.4f} ± {np.std(baseline_results['times']):.4f} s")

print("\n2. Q-LEARNING COM RANDOM WALK (20000 passos)")
print("-" * 80)
print(f"   Recompensa final no teste: {np.mean(random_walk_results['test_rewards'][:, -1]):.4f} ± {np.std(random_walk_results['test_rewards'][:, -1]):.4f}")
print(f"   Tempo médio de treino: {np.mean(random_walk_results['execution_times']):.2f} ± {np.std(random_walk_results['execution_times']):.2f} s")

print("\n3. ε-GREEDY (20000 passos)")
print("-" * 80)
for greed in greed_values:
    mean_reward = np.mean(epsilon_greedy_results[greed]['test_rewards'][:, -1])
    std_reward = np.std(epsilon_greedy_results[greed]['test_rewards'][:, -1])
    mean_time = np.mean(epsilon_greedy_results[greed]['execution_times'])
    std_time = np.std(epsilon_greedy_results[greed]['execution_times'])
    print(f"   greed={greed}: Recompensa = {mean_reward:.4f} ± {std_reward:.4f}, Tempo = {mean_time:.2f} ± {std_time:.2f} s")

print("\n" + "="*80)
print("OBSERVAÇÕES:")
print("="*80)
print("1. As paredes tornam o problema mais difícil, aumentando o número médio de passos")
print("2. A penalização de -0.1 ajuda o agente a evitar bater em paredes repetidamente")
print("3. Estratégias com maior exploração (greed baixo) podem ser mais eficazes")
print("   em ambientes com paredes, pois descobrem caminhos alternativos")
print("4. O Q-Learning aprende a navegar eficientemente mesmo com obstáculos")
print("="*80)

## 7. Análise do Efeito da Penalização

A penalização de -0.1 ao bater em paredes tem os seguintes efeitos:

1. **Desencoraja ações inúteis**: O agente aprende a evitar ações que levam a paredes
2. **Acelera a aprendizagem**: A penalização fornece feedback negativo imediato
3. **Melhora a navegação**: O agente aprende caminhos mais eficientes
4. **Equilibra exploração/exploração**: A penalização não é tão grande que impeça exploração

In [ ]:
# Análise detalhada da trajetória do agente treinado
def analyze_trajectory(agent, env, initial_state=1, max_steps=1000):
    """Analisa trajetória do agente treinado."""
    trajectory = [initial_state]
    state = initial_state
    wall_hits = 0
    
    for _ in range(max_steps):
        action = agent.get_best_action(state)
        next_state, reward = env.transition(state, action)
        
        # Conta batidas em parede
        if next_state == state and reward == env.wall_penalty:
            wall_hits += 1
        
        trajectory.append(next_state)
        
        if next_state == env.goal_state:
            break
        
        state = next_state
    
    return trajectory, wall_hits

# Analisa melhor agente
np.random.seed(10)
best_agent = QLearningAgentWithWalls(env)
best_agent.train_epsilon_greedy(n_training_steps, test_points, greed=0.9)

trajectory, wall_hits = analyze_trajectory(best_agent, env)

print("\nANÁLISE DA TRAJETÓRIA DO MELHOR AGENTE")
print("="*60)
print(f"Número de passos até o objetivo: {len(trajectory) - 1}")
print(f"Número de batidas em parede: {wall_hits}")
print(f"Estados visitados: {trajectory[:20]}..." if len(trajectory) > 20 else f"Estados visitados: {trajectory}")
print("="*60)

In [ ]:
# Visualizar trajetória
def visualize_trajectory(env, trajectory):
    """Visualiza trajetória do agente."""
    grid = np.zeros((env.grid_size, env.grid_size))
    
    # Marca paredes
    for wall in env.walls:
        row, col = env.state_to_position(wall)
        grid[row, col] = -1
    
    # Marca trajetória
    for i, state in enumerate(trajectory):
        if state not in env.walls:
            row, col = env.state_to_position(state)
            grid[row, col] = i + 1
    
    plt.figure(figsize=(12, 10))
    plt.imshow(grid, cmap='coolwarm', interpolation='nearest')
    plt.colorbar(label='Ordem de visita')
    
    # Adiciona setas para mostrar direção
    for i in range(len(trajectory) - 1):
        state = trajectory[i]
        next_state = trajectory[i + 1]
        
        if state != next_state and state not in env.walls:
            row1, col1 = env.state_to_position(state)
            row2, col2 = env.state_to_position(next_state)
            
            if i % 3 == 0:  # Mostrar apenas algumas setas para clareza
                plt.arrow(col1, row1, (col2 - col1) * 0.4, (row2 - row1) * 0.4,
                         head_width=0.2, head_length=0.15, fc='yellow', ec='yellow', linewidth=2)
    
    plt.title('Trajetória do Agente Treinado (Com Paredes)', fontsize=14, weight='bold')
    plt.tight_layout()
    plt.show()

visualize_trajectory(env, trajectory)

## 8. Conclusões

### Principais Descobertas:

1. **Impacto das Paredes**:
   - As paredes aumentam significativamente a dificuldade do problema
   - O número médio de passos para atingir o objetivo aumenta
   - A navegação requer aprendizagem de caminhos mais complexos

2. **Efeito da Penalização**:
   - A penalização de -0.1 é eficaz para desencorajar batidas em paredes
   - Não é tão grande a ponto de dominar o sinal de recompensa do objetivo
   - Ajuda o agente a aprender políticas mais eficientes

3. **Estratégias de Aprendizagem**:
   - ε-greedy com greed moderado (0.5-0.7) é eficaz em ambientes com obstáculos
   - Muito exploitation (greed = 0.9) pode levar a políticas subótimas
   - Random walk puro é muito ineficiente neste ambiente

4. **Comparação com Ambiente Sem Paredes**:
   - O Q-Learning ainda é capaz de aprender boas políticas
   - Requer mais passos de treino para convergir
   - A exploração é mais importante neste cenário

## 9. Salvando Resultados

In [ ]:
# Salvar resultados em arquivo
import pickle

results_to_save = {
    'baseline': baseline_results,
    'random_walk': random_walk_results,
    'epsilon_greedy': epsilon_greedy_results,
    'test_points': test_points,
    'greed_values': greed_values,
    'environment': {
        'walls': list(env.walls),
        'wall_penalty': env.wall_penalty,
        'goal_reward': env.goal_reward
    }
}

with open('exercicio4_results.pkl', 'wb') as f:
    pickle.dump(results_to_save, f)

print("Resultados salvos em 'exercicio4_results.pkl'")